In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/diagnostic_ability/auc_comparison.csv'
sheet = None

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/diagnostic_ability'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
df = cal_palm.read_and_display_data()

In [ ]:
import pandas as pd
# df = pd.read_csv(csv_path)
# df = df[df['Method'] != 'All']
# df = df.loc[0:3, :]
# df = df.loc[:, ~df.columns.str.contains('_P')]
df

In [ ]:
df.pop('AtrophyMethod')
df.pop('Smooth')

Simple Bar Plot

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

def plot_barplots(df, x_col, y_col, hue=None, palette="tab10"):
    ax = sns.barplot(data=df, x=x_col, y=y_col, hue=hue, palette=palette)
    return ax 
    
def plot_point_as_diamond(df, ax, point_col, palette="tab10"):
    x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches]
    plt.scatter(x=x_coords, y=df[point_col], color='k', marker='D', facecolors='none')
    
def plot_error_bars(df, ax, err_col, palette="tab10"):
    x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches]
    y_coords = [p.get_height() for p in ax.patches]
    ax.errorbar(x=x_coords, y=y_coords, yerr=df[err_col], capsize=6, fmt="none", c="k")

def show_and_save_plot(out_dir=None):
    if out_dir is not None:
        plt.savefig(os.path.join(out_dir, 'barplot.svg'))
    plt.show()

In [ ]:
# Call the plotting functions
plt.figure(figsize=(10,6))
ax = plot_barplots(df, 'Regression_Method', 'Point_R_squared')
# plot_error_bars(df=df, ax=ax, err_col='Std')
# plot_point_as_diamond(df=df, ax=ax, point_col='Point_R_squared')
show_and_save_plot(out_dir)

Create Barplot Of Methods Within Groups
- The groups to compare within are the top level keys of the dictionary. 
- The groups to compare across are the second-level keys

In [ ]:
df

In [ ]:
import pandas as pd

def melt_dataframe(df):
    # Melt the DataFrame
    melted_df = pd.melt(df, id_vars=['Segment'], var_name='Validation', value_name='AUROC')
    
    # Extract the region names from the 'Region' column
    melted_df['Segment'] = melted_df['Segment'].str.split(' ', expand=True)[0]
    
    return melted_df

melted_df = melt_dataframe(df)
melted_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Create the bar plot
plt.figure(figsize=(12, 6))
sns.set_palette("tab10")
sns.barplot(x='Validation', y='AUROC', hue='Segment', data=melted_df)

# Customizing the plot
plt.ylim(0.5, 1)
plt.title('AUROC')
plt.ylabel('AUROC')
plt.xlabel('Region')
output_file_path = os.path.join(out_dir, 'barplot.svg')
plt.savefig(output_file_path, format='svg')
# Show the plot
plt.show()

Swap Grouping

In [ ]:
df


In [ ]:
# Transpose the data to swap grouping
df = pd.DataFrame(df).T.melt(var_name='Inference', value_name='Accuracy', ignore_index=False).reset_index()

# Generate Grouped Barplot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Create the bar plot with swapped grouping
plt.figure(figsize=(12, 6))
sns.set_palette("tab10")
sns.barplot(x='Method', y='R-Squared', hue='Variable', data=melted_df)

# Customizing the plot
# plt.ylim(-0.3, 0.8)  # Adjust the y-axis limits as needed
plt.title('Agreement With Radiologist')
plt.xlabel('Method')
plt.ylabel('Correlation to Neuroradiologist (Spearman Rho)')

# Save the plot as an SVG file
output_file_path = os.path.join(out_dir, 'barplot.svg')
# plt.savefig(output_file_path, format='svg')

# Show the plot (optional)
plt.show()


# Generate Boxplot

In [ ]:
# df.drop(13, inplace=True)
# df.drop(17, inplace=True)
# df.drop(14, inplace=True)


In [ ]:
melted_df.loc[8, 'AUROC'] = 0.7

In [ ]:
melted_df


In [ ]:
# df = pd.DataFrame(df).T.melt(var_name='Imaging Method', value_name='R-Squared', ignore_index=False).reset_index()
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
# Create the bar plot with swapped grouping
plt.figure(figsize=(12, 6))
sns.set_palette("tab10")
sns.boxplot(x='Segment', y='AUROC', data=melted_df)

# Customizing the plot
# plt.ylim(-0.3, 0.8)  # Adjust the y-axis limits as needed
plt.title('AUROC by Method')
plt.xlabel('')
plt.ylabel('AUROC')
plt.ylim((0.5,1.0))

# Save the plot as an SVG file
output_file_path = os.path.join(out_dir, 'boxplot_v2.svg')
plt.savefig(output_file_path, format='svg')
# Show the plot (optional)
plt.show()

Test the Box PLot

In [ ]:
melted_df.columns

In [ ]:
from calvin_utils.statistical_utils.kruskall_wallis import KruskalWallisTest
kwt = KruskalWallisTest(data=melted_df, value_column='AUROC', group_column='Segment')

# Stacked barplot

In [ ]:
data_df.columns

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os

def plot_stacked_barplots(df, columns, out_dir):
    # Create a DataFrame to hold the counts for each response category for each question
    response_categories = df[columns].apply(pd.Series.value_counts).fillna(0).T

    # Normalize the counts to be on a scale of 0-1
    response_categories = response_categories.div(response_categories.sum(axis=1), axis=0)
    
    # Ensure consistent coloring using tab10
    cmap = plt.get_cmap('tab10')
    unique_responses = response_categories.columns
    colors = {response: cmap(i) for i, response in enumerate(unique_responses)}

    # Plotting the stacked bar plot
    ax = response_categories.plot(kind='bar', stacked=True, figsize=(20, 20), color=[colors[response] for response in unique_responses])

    # Annotating the plot with the normalized values
    for i in range(len(response_categories)):
        cumulative_sum = np.zeros(len(response_categories.columns))
        for j, response in enumerate(response_categories.columns):
            height = response_categories.iloc[i, j]
            cumulative_sum[j] = cumulative_sum[j-1] + height if j > 0 else height
            ax.text(i, cumulative_sum[j] - height / 2, f'{height:.2f}', ha='center', va='center', fontsize=12, color='black')

    # Customizing the plot
    plt.title('Normalized Stacked Bar Plot of Responses', fontsize=20)
    plt.ylabel('Proportion of Responses', fontsize=16)
    plt.legend(title='Responses', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)
    plt.xticks(range(len(columns)), [col.replace('_', ' ') for col in columns], rotation=45, ha='right', fontsize=16)
    plt.yticks(fontsize=16)
    sns.despine()
    plt.tight_layout()
    
    if out_dir is not None:
        plt.savefig(os.path.join(out_dir, 'stacked_barplot.svg'))
    
    plt.show()

In [ ]:
# Likert Questions:
# columns_to_plot = [
#     'How_intuitive_was_the_software_assessment_compared_to_the_paper_test_',
#     'How_comfortable_were_you_navigating_through_the_software_',
#     'How_confident_do_you_feel_being_able_to_complete_the_test_on_your_own_',
#     'How_much_additional_value_do_you_see_this_assessment_bringing_to_your_medical_care_'
# ]
# Yes/No Questions:
columns_to_plot = [
    'Would_you_find_it_useful_if_you_could_take_the_test_before_your_appointment_and_talk_about_the_results_with_your_doctor_',
       'Would_you_recommend_this_assessment_to_someone_else_',
       'Would_being_able_to_take_this_test_remotely_help_your_care_',
       'Would_taking_this_test_alone_alleviate_the_stress_of_being_assessed_by_a_doctor_',
       'Would_taking_this_test_alone_alleviate_feelings_of_judgement_during_a_test_',
        'Would_you_trust_the_results_of_this_test_'
]
# Assuming 'df' is your DataFrame
plot_stacked_barplots(data_df, columns_to_plot, out_dir)


# 05 - Scatterplot a Value to a Classification